# Simulating Self-Exciting Point Processes (SEPP) with Random Kernels
This notebook simulates spatial-temporal point processes using custom kernels and stores the resulting event data.
It follows a structured approach: kernel definitions, sampling, and data storage.

## 0. Import and path configuration
Sets up the environment, loads libraries and sets Python paths for module imports.

In [ ]:
import sys, os.path, os
from pathlib import Path

# Set relative paths to required libraries (update according to your local folder structure)
path_opencp = Path('Librerias/PredictCode')
path_fairness = Path('Librerias')
path_exp = Path('Notebooks_for_topic/EXP0/SCRIPTS')

# Add paths to system path
sys.path.insert(0, str(path_opencp.resolve()))
sys.path.insert(0, str(path_fairness.resolve()))
sys.path.insert(0, str(path_exp.resolve()))

## 1. Load libraries and define helper plotting functions
Imports required packages and defines visualization functions for event data.

In [ ]:
import open_cp
import numpy as np
import open_cp.plot as plot
import matplotlib.pyplot as plt
import datetime
import pickle as pkl

# Plot raw spatial and temporal events
def plot_events(data, f_inicial, f_final):
    fig, ax = plt.subplots(ncols=2, figsize=(9,2.5))
    ax[0].scatter(data.xcoords, data.ycoords, marker='+', alpha=0.5)
    ax[0].set_title('Coordinates')
    ax[0].set_xlabel('Coordinate X')
    ax[0].set_ylabel('Coordinate Y')
    ax[0].set(xlim=[0,1], ylim=[0,1])
    
    times = data.times_datetime()
    ax[1].scatter(times, data.ycoords, marker='+', alpha=0.1)
    ax[1].set_xlim([f_inicial, f_final])
    ax[1].set_ylim([0, 1])
    ax[1].set_title('Coordinate Vs Dates')
    ax[1].set_xlabel('Date')
    ax[1].set_ylabel('Coordinate Y')
    fig.autofmt_xdate()

## 2. Setup simulation parameters
Defines region, time units and other global parameters for SEPP simulation.

In [ ]:
from open_cp.sources.sepp import make_time_unit
from global_vars import x_min, x_max, y_min, y_max, f_inicial, f_final, grid_size, days_time_unit, n_events_day

region = open_cp.RectangularRegion(x_min, x_max, y_min, y_max)
time_unit = make_time_unit(datetime.timedelta(days=days_time_unit))
total_days = (f_final - f_inicial).days
number_of_samples = 30

# Load additional components
from open_cp.sources.random import KernelSampler
from open_cp.sources.sepp import Exponential, HomogeneousPoisson, InhomogeneousPoissonFactors
import open_cp.naive as naive

## 3. Define SEPP sampling function
Generates a sample using spatial and temporal kernels.

In [ ]:
from open_cp.kernels import GaussianKernel

def sepp_sample(means_hotspots, variances_hotspots, total_rate_exp, variance_trigg, rate_time_kernel, f_inicial, time_unit, region, **args):
    GK = GaussianKernel(means_hotspots.T, variances_hotspots.T)
    try:
        GK.set_scale(1 / max(GK(means_hotspots.T)))
    except:
        GK.set_scale(1 / GK(means_hotspots.T))
    
    K_sampler = KernelSampler(region, GK, 1)
    Homo_timeK = HomogeneousPoisson(rate=rate_time_kernel)
    Exp_timeK = Exponential(1, total_rate_exp)

    background_sampler = InhomogeneousPoissonFactors(Homo_timeK, K_sampler)
    trig_space_sampler = open_cp.sources.sepp.GaussianSpaceSampler([0,0], [variance_trigg, variance_trigg], 0)
    trigger_sampler = InhomogeneousPoissonFactors(Exp_timeK, trig_space_sampler)

    simulation = open_cp.sources.sepp.SelfExcitingPointProcess(background_sampler, trigger_sampler)
    sample = simulation.sample(0, total_days)
    timed_points = open_cp.sources.sepp.scale_to_real_time(sample, f_inicial, time_unit)
    return timed_points

## 4. Generate random kernel parameters
Generates random spatial and temporal kernel parameters for simulation.

In [ ]:
def random_vars():
    n_spatial_hotspots = np.random.randint(1, 5)
    centros_hotspots = np.random.rand(n_spatial_hotspots, 2)
    varianzas = np.random.rand(n_spatial_hotspots, 2) * 0.04
    exp_rate = np.random.rand() * 0.05 + 0.7
    trigger_var = np.random.rand() * 0.001
    
    return {
        "means_hotspots": centros_hotspots,
        "variances_hotspots": varianzas,
        "total_rate_exp": exp_rate,
        "variance_trigg": trigger_var,
        "rate_time_kernel": n_events_day / 3,
        "n_spatial_hotspots": n_spatial_hotspots
    }

## 5. Simulation loop and data storage
Runs the simulation multiple times and saves the resulting datasets.

In [ ]:
from global_vars import dir_sims
dir_parametros = os.path.join(dir_sims, "parametros")
prefix_data = "Data_"

for i in range(number_of_samples):
    parms = random_vars()
    parms["f_inicial"] = f_inicial
    parms["time_unit"] = time_unit
    parms["region"] = region

    timed_points = sepp_sample(**parms)
    print(timed_points.number_data_points)
    plot_events(timed_points, f_inicial, f_final)
    pkl.dump(timed_points, open(os.path.join(dir_sims, prefix_data + str(i) + ".pkl"), "wb"))